In [2]:
import pandas as pd


## version1


In [3]:
def deter3And5Utr(sequence):
    Utrindex={
    "5'start_index":[],
    "5'end_index":[],
    "3'start_index":[],
    "3'end_index":[],
    }
    if sequence[0].islower():
        for i,char in enumerate(sequence):
            if char.isupper():
                Utrindex["5'start_index"].append(1)
                Utrindex["5'end_index"].append(i)
                break
    else:
        Utrindex["5'start_index"].append(None)
        Utrindex["5'end_index"].append(None)
    
    if len(sequence) > 0 and sequence[-1].islower():
        for i in range(len(sequence)-1):
            if sequence[i].isupper() and sequence[i + 1].islower():
                Utrindex["3'start_index"].append(i+1)
                Utrindex["3'end_index"].append(len(sequence))
    else:
        Utrindex["3'start_index"].append(None)
        Utrindex["3'end_index"].append(None) 
    # if len(Utrindex["3'start_index"]) > 0:
    #     Utrindex["3'start_index"] = [Utrindex["3'start_index"][-1]]
    #     Utrindex["3'end_index"] = [Utrindex["3'end_index"][-1]]
    return Utrindex
        

## version2


In [4]:
def transform(c:str):
    if c.isupper():
        return"big"
    else:
        return"small"

In [5]:
def chips(sequence):
    prev_state=None
    i=0
    start=0
    result=[]
    while True:
        if i==len(sequence):
            break
        cur=transform(sequence[i])
        if prev_state==None:
            prev_state=cur
        else:
            if prev_state!=cur:
                result.append(sequence[start:i])
                prev_state=cur
                start=i
        i+=1
    result.append(sequence[start:])
    return result


In [14]:
def typename(splice:list[str]):
    hw2_list=[]
    start=0
    for i in range(len(splice)):
        length=len(splice[i])
        end=start+len(splice[i])
        info=splice[i]
        hw2_list.append([start+1,end,length,info])
        start=start+length
    start=0
    end=len(splice)
    if splice[0].islower():
        hw2_list[1][0]=hw2_list[0][0]
        hw2_list[1][2]+=hw2_list[0][2]
        hw2_list[0]=["5'utr"]+hw2_list[0]
        start=1
    if splice[-1].islower():
        hw2_list[-2][1]=hw2_list[-1][1]
        hw2_list[-2][2]+=hw2_list[-1][2]
        hw2_list[-1]=["3'utr"]+hw2_list[-1]
        end-=1
    exon_index=1
    intron_index=1
    for i,_ in enumerate(hw2_list[start:end]):
        if splice[i+start].isupper():
            hw2_list[i+start]=[f"Exon{exon_index}"]+ hw2_list[i+start]
            exon_index+=1
        if splice[i+start].islower():
            hw2_list[i+start]=[f"Intron{intron_index}"]+ hw2_list[i+start]
            intron_index+=1
    return hw2_list
        


In [7]:
with open('raw_data/unspliced+UTRTranscriptSequence_Y40B10A.2a.1 (1).fasta','r')as file:
    both=file.readlines()
with open('raw_data/without3\'.fasta','r')as file:
    without3=file.readlines()
with open('raw_data/without5\'.fasta','r')as file:
    without5=file.readlines()
with open('raw_data/withoutboth.fasta','r')as file:
    withoutboth=file.readlines()
    

In [15]:
test=chips(both[1])

y=typename(test)
y


[["5'utr",
  1,
  131,
  131,
  'aagagacgcagaaatcacaatgtttgtcatcgtttttttcttgttctctcggactatctgatagtataaattcttccctcttttgcctccccaatccaattaattttcagtttttttttccgaaaaaaaaa'],
 ['Exon1',
  1,
  191,
  191,
  'ATGTCCAACGTCGCAAAAAGCTACAAGAGCTCGGATCCAGTGATTGCGTATTGCTCGGAG'],
 ['Intron1',
  192,
  2028,
  1837,
  'gtttgtagtttgtagtttgtagtttgtaatgtcacggatttctggtttccttcataaattgaaatggaagagtttgccgaactaggccattttggctcggccatatctgttgtagatttacggcgcgttgcgtgtcgcgtcgcggttcgagtttagttgttaaactaaatgtatttgtccgtgtggagtacacgactttcccacgcgttgtccggcgggcgattgtcaatggagcgcgaaaaattcaatgaggaaggcccacggtagcacagaaactagatgggttggtttatgatactcagtctggaaacctatattggctactatctccaaaaccatcacaaaatcgattttgcgatgcatataagaaatgaccgcaatgaaattatctatctttatttgtgatgaattttcgattttgtacttcctggccaagttatgcacgtttgttcgatggagcgcgtttgaacccatctagcaactgaaccaccgtgggccagaaccccccgtgaatcgtcgtggaaacgcgctccgcctccaacagttgggtctcactaggtaaaaatttgggcggcaaaattgtaaattcaaatttttttaccaattttcaagcattttcttgaagtttcagaagaaaaataaaataaacatttttttttcgatttttcaatattgcaaattcacataattctaaaa